In [1]:
import urllib.request
import json
import time
import datetime

In [4]:
def get_FX_from_gaitame(currencyPairCode="USDJPY"):
    """
    外為オンラインよりask/bidを取得する。
    
    ＜メソッドを作成するにあたって参考にしたHP＞
        http://qiita.com/chromabox/items/a1323225bae146c80bec
    
    ＜注意事項＞
        １秒ごとに更新されるが、このメソッドで１秒ごとに取得することはネチケット上やめるべき。
        最低でも２秒は空けたい。
    
    ＜入力＞
        currencyPairCode: 外貨ペア
    
    ＜出力＞
        nowtime: リクエストしたときの日時
        ask: 対象とする外貨ペアのask
        bid: 対象とする外貨ペアのbid
    """
    ask = -1.0; bid = -1.0; is_gotten = False
    url = "http://www.gaitameonline.com/rateaj/getrate"
    nowtime = datetime.datetime.now()
    try:
        res = urllib.request.urlopen(url)
        quotes = json.loads(res.read().decode('utf-8'))["quotes"]
        for quo in quotes:
            if quo["currencyPairCode"] == currencyPairCode: 
                try:
                    ask = float(quo["ask"])
                    bid = float(quo["bid"])
                    is_gotten = True
                except:
                    ask = -1.0; bid = -1.0; is_gotten = False
                break
    except Exception as e:
        print(nowtime, e)
    return nowtime, ask, bid, is_gotten

def ToDataFrame(nts, fxs):
    """
    日時と値からデータフレームを生成する
    ＜入力＞
        nts: 日時（リスト）
        fxs: データ（[ask, bid]のリスト）
    """
    df = pd.DataFrame(fx, columns=["ask", "bid"])
    df.index = nowtimes
    return df

def is_weekend(dt):
    """
    Judge whether the day is weekend.
    Here `weekend` correspoonds to the range from 06:00:00 on Saturday to 06:00:00 on Monday.
    """
    if dt.weekday() == 5:
        return dt.hour >= 6
    elif dt.weekday() == 6:
        return True
    elif dt.weekday() == 0:
        return dt.hour < 6
    else:
        return False

In [ ]:
update_time = 300 # この時間ごとに更新 [sec]
update_count = 12 # この回数だけ取得したらリストを初期化する
savenow = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
savepath = "../data/fxdata_{0}.csv".format(savenow)

st = time.time()
nowtimes = []
fx = []
count = 0

nowtime = datetime.datetime.now()
isholiday = is_weekend(nowtime)
if isholiday:
    print(nowtime.strftime("%Y%m%d_%H%M%S"), "holiday.")
    print("Wait until holiday ends...")
while True:
    try:
        if not isholiday:
            nowtime, ask, bid, is_gotten = get_FX_from_gaitame()
            if is_weekend(nowtime):
                print("holiday.")
                print("Wait until holiday ends...")
                isholiday = True
                if len(nowtimes) != 0:
                    df = ToDataFrame(nowtimes, fx)
                    df.to_csv(savepath)
                nowtimes = []
                fx = []
                count = 0
                savenow = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
                savepath = "../data/fxdata_{0}.csv".format(savenow)
                time.sleep(60)
                continue
        else:
            if not is_weekend(datetime.datetime.now()):
                print("holiday ends.")
                savenow = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
                savepath = "../data/fxdata_{0}.csv".format(savenow)
                isholiday = False
                continue
            else:
                time.sleep(60)
                continue
        if not is_gotten:
            print(datetime.datetime.now().strftime("%Y%m%d_%H%M%S"), "Failure in getting FX data.")
            time.sleep(5.0)
            continue
        nowtimes.append(nowtime.strftime("%Y-%m-%d %H:%M:%S"))
        fx.append([ask, bid])
        time.sleep(5.0)
        if time.time() - st > update_time:
            st = time.time()
            df = ToDataFrame(nowtimes, fx)
            df.to_csv(savepath)
            count += 1
            if count >= update_count:
                nowtimes = []
                fx = []
                savenow = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
                savepath = "../data/fxdata_{0}.csv".format(savenow)
                count = 0
    except Exception as e:
        print(datetime.datetime.now().strftime("%Y%m%d_%H%M%S"))
        print(e)
        df = ToDataFrame(nowtimes, fx)
        df.to_csv(savepath)
        print("Wait for 30 second...")
        time.sleep(30) # 失敗したときは３０秒ほど空ける
        nowtimes = []
        fx = []
        savenow = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        savepath = "../data/fxdata_{0}.csv".format(savenow)
        

20170527_210138 holiday.
Wait until holiday ends...
holiday ends.
2017-05-29 07:27:59.943597 <urlopen error [WinError 10060] 接続済みの呼び出し先が一定の時間を過ぎても正しく応答しなかったため、接続できませんでした。または接続済みのホストが応答しなかったため、確立された接続は失敗しました。>
20170529_072820 Failure in getting FX data.
2017-05-29 13:41:29.964361 <urlopen error [WinError 10060] 接続済みの呼び出し先が一定の時間を過ぎても正しく応答しなかったため、接続できませんでした。または接続済みのホストが応答しなかったため、確立された接続は失敗しました。>
20170529_134149 Failure in getting FX data.
